This notebook is based on the tensorflow docs: 


https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams

adapted for a regression problem.

In [1]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

In [2]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import numpy as np
from sklearn.preprocessing import StandardScaler

In [4]:
    file = "/home/isidro/chains_server/owaCDM_phy_HD+SN+BBAO+Planck_nested_dynesty_multi_1.txt"

    split = 0.2
    epochs = 100
    nlayers = 2
    batch_size = 0

    params = np.loadtxt(file, usecols=(2,3,4,5,6))
    logL = np.loadtxt(file, usecols=(1))

    nparams = len(params)
    randomize = np.random.permutation(nparams)
    params = params[randomize]
    logL = logL[randomize]

    maxLogL = np.max(logL)
    minLogL = np.min(logL)
    ntrain = int(split * nparams)
    indx = [ntrain]
    # xtrain <- params_training ytrain<-logLtraining
    params_training, params_testing = np.split(params, indx)
    logL_training, logL_testing = np.split(logL, indx)

#     sc_X = StandardScaler()
#     X_train = sc_X.fit_transform(X_train)
#     X_test = sc_X.transform(X_test)

    print("Total len dataset {}," 
          "len training set {}, len test set {}".format(nparams, 
                                                     len(params_training), len(params_testing)))

Total len dataset 22267,len training set 4453, len test set 17814


In [5]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',
                                   min_delta=1e-4,
                                   patience=5,
                                   restore_best_weights=True)
                                   ]

n_cols = params_training.shape[1]

In [6]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([100, 200, 300, 400, 500]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([2,8,16,32,64]))


In [7]:
# METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_OPTIMIZER, HP_BATCHSIZE],
    metrics=[hp.Metric('loss', display_name="Loss")])  


In [8]:
def train_test_model(hparams):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu, input_shape=(n_cols,)),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        tf.keras.layers.Dense(1)
    ])
                              
    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='mean_squared_error',
        metrics=['mean_squared_error'],
      )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    # _, loss = model.evaluate(x_test, y_test)
    model.fit(params_training, logL_training, epochs=epochs, 
              batch_size=hparams[HP_BATCHSIZE], callbacks=callbacks) 
    _, loss = model.evaluate(params_testing, logL_testing)
    
    
    return loss

In [9]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  # record the values used in this trial
    loss = train_test_model(hparams)
    tf.summary.scalar("loss", loss, step=1)


In [10]:
session_num = 0

for batch_size in HP_BATCHSIZE.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
      for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:
          hparams = {
              HP_NUM_UNITS: num_units,
              HP_DROPOUT: dropout_rate,
              HP_OPTIMIZER: optimizer,
              HP_BATCHSIZE: batch_size
          }
          run_name = "run-%d" % session_num
          print('--- Starting trial: %s' % run_name)
          print({h.name: hparams[h] for h in hparams})
          run('logs/hparam_tuning/' + run_name, hparams)
          session_num += 1


--- Starting trial: run-0
{'num_units': 100, 'dropout': 0.0, 'optimizer': 'Adadelta', 'batch_size': 2}
Epoch 1/100
2227/2227 [==============================] - 2s 989us/step - loss: 6337555456.0000 - mean_squared_error: 6337555456.0000
Epoch 2/100
2227/2227 [==============================] - 2s 1ms/step - loss: 6337549312.0000 - mean_squared_error: 6337549312.0000
Epoch 3/100
2227/2227 [==============================] - 2s 1ms/step - loss: 6337551872.0000 - mean_squared_error: 6337551872.0000
Epoch 4/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337550336.0000 - mean_squared_error: 6337550336.0000
Epoch 5/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337547776.0000 - mean_squared_error: 6337547776.0000
Epoch 6/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337550848.0000 - mean_squared_error: 6337550848.0000
Epoch 7/100
2227/2227 [==============================] - 2s 1ms/step - loss: 6337552896.0000 - mean_squared_e

2227/2227 [==============================] - 4s 2ms/step - loss: 1405254016.0000 - mean_squared_error: 1405254016.0000
Epoch 51/100
557/557 [==============================] - 1s 2ms/step - loss: 2567992064.0000 - mean_squared_error: 2567992064.0000
--- Starting trial: run-2
{'num_units': 100, 'dropout': 0.2, 'optimizer': 'Adadelta', 'batch_size': 2}
Epoch 1/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337555456.0000 - mean_squared_error: 6337555456.0000
Epoch 2/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337556992.0000 - mean_squared_error: 6337556992.0000
Epoch 3/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337554432.0000 - mean_squared_error: 6337554432.0000
Epoch 4/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337555456.0000 - mean_squared_error: 6337555456.0000
Epoch 5/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337552896.0000 - mean_squared_error: 633755289

2227/2227 [==============================] - 2s 1ms/step - loss: 1567016320.0000 - mean_squared_error: 1567016320.0000
Epoch 43/100
557/557 [==============================] - 0s 811us/step - loss: 2759615744.0000 - mean_squared_error: 2759615744.0000
--- Starting trial: run-4
{'num_units': 200, 'dropout': 0.0, 'optimizer': 'Adadelta', 'batch_size': 2}
Epoch 1/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337548800.0000 - mean_squared_error: 6337548800.0000
Epoch 2/100
2227/2227 [==============================] - 3s 2ms/step - loss: 6337553920.0000 - mean_squared_error: 6337553920.0000
Epoch 3/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337548288.0000 - mean_squared_error: 6337548288.0000
Epoch 4/100
2227/2227 [==============================] - 3s 2ms/step - loss: 6337548800.0000 - mean_squared_error: 6337548800.0000
Epoch 5/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337548800.0000 - mean_squared_error: 6337548

2227/2227 [==============================] - 4s 2ms/step - loss: 6337549312.0000 - mean_squared_error: 6337549312.0000
Epoch 6/100
2227/2227 [==============================] - 3s 2ms/step - loss: 6337548800.0000 - mean_squared_error: 6337548800.0000
Epoch 7/100
2227/2227 [==============================] - 3s 2ms/step - loss: 6337549312.0000 - mean_squared_error: 6337549312.0000
Epoch 8/100
2227/2227 [==============================] - 3s 2ms/step - loss: 6337549312.0000 - mean_squared_error: 6337549312.0000
Epoch 9/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337548288.0000 - mean_squared_error: 6337548288.0000
Epoch 10/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337551360.0000 - mean_squared_error: 6337551360.0000
Epoch 11/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337550336.0000 - mean_squared_error: 6337550336.0000
Epoch 12/100
2227/2227 [==============================] - 3s 1ms/step - loss: 6337547264.0000

2227/2227 [==============================] - 4s 2ms/step - loss: 6337548288.0000 - mean_squared_error: 6337548288.0000
Epoch 5/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337550848.0000 - mean_squared_error: 6337550848.0000
Epoch 6/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337548288.0000 - mean_squared_error: 6337548288.0000
Epoch 7/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337542656.0000 - mean_squared_error: 6337542656.0000
Epoch 8/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337549312.0000 - mean_squared_error: 6337549312.0000
Epoch 9/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337538560.0000 - mean_squared_error: 6337538560.0000
Epoch 10/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337538560.0000 - mean_squared_error: 6337538560.0000
Epoch 11/100
2227/2227 [==============================] - 4s 2ms/step - loss: 6337538560.0000 

...

70/70 [==============================] - 0s 5ms/step - loss: 1645405056.0000 - mean_squared_error: 1645405056.0000
Epoch 19/100
70/70 [==============================] - 0s 5ms/step - loss: 1637538688.0000 - mean_squared_error: 1637538688.0000
Epoch 20/100
70/70 [==============================] - 0s 5ms/step - loss: 1629421952.0000 - mean_squared_error: 1629421952.0000
Epoch 21/100
70/70 [==============================] - 0s 5ms/step - loss: 1628120576.0000 - mean_squared_error: 1628120576.0000
Epoch 22/100
70/70 [==============================] - 0s 5ms/step - loss: 1610373120.0000 - mean_squared_error: 1610373120.0000
Epoch 23/100
70/70 [==============================] - 0s 5ms/step - loss: 1615990400.0000 - mean_squared_error: 1615990400.0000
Epoch 24/100
70/70 [==============================] - 0s 5ms/step - loss: 1595183616.0000 - mean_squared_error: 1595183616.0000
Epoch 25/100
70/70 [==============================] - 0s 5ms/step - loss: 1591193728.0000 - mean_squared_error: 15911

70/70 [==============================] - 0s 6ms/step - loss: 1389814400.0000 - mean_squared_error: 1389814400.0000
Epoch 83/100
70/70 [==============================] - 0s 6ms/step - loss: 1382499584.0000 - mean_squared_error: 1382499584.0000
Epoch 84/100
70/70 [==============================] - 0s 6ms/step - loss: 1386681088.0000 - mean_squared_error: 1386681088.0000
Epoch 85/100
70/70 [==============================] - 0s 7ms/step - loss: 1388368896.0000 - mean_squared_error: 1388368896.0000
Epoch 86/100
70/70 [==============================] - 0s 6ms/step - loss: 1380795136.0000 - mean_squared_error: 1380795136.0000
Epoch 87/100
70/70 [==============================] - 0s 6ms/step - loss: 1374513664.0000 - mean_squared_error: 1374513664.0000
Epoch 88/100
70/70 [==============================] - 0s 7ms/step - loss: 1386583040.0000 - mean_squared_error: 1386583040.0000
Epoch 89/100
70/70 [==============================] - 0s 6ms/step - loss: 1369452288.0000 - mean_squared_error: 13694

70/70 [==============================] - 0s 5ms/step - loss: 1615514752.0000 - mean_squared_error: 1615514752.0000
Epoch 31/100
70/70 [==============================] - 0s 5ms/step - loss: 1612518912.0000 - mean_squared_error: 1612518912.0000
Epoch 32/100
70/70 [==============================] - 0s 5ms/step - loss: 1538761600.0000 - mean_squared_error: 1538761600.0000
Epoch 33/100
70/70 [==============================] - 0s 5ms/step - loss: 1579764864.0000 - mean_squared_error: 1579764864.0000
Epoch 34/100
70/70 [==============================] - 0s 5ms/step - loss: 1564544768.0000 - mean_squared_error: 1564544768.0000
Epoch 35/100
70/70 [==============================] - 0s 5ms/step - loss: 1515648000.0000 - mean_squared_error: 1515648000.0000
Epoch 36/100
70/70 [==============================] - 0s 5ms/step - loss: 1562946688.0000 - mean_squared_error: 1562946688.0000
Epoch 37/100
70/70 [==============================] - 0s 5ms/step - loss: 1552491776.0000 - mean_squared_error: 15524

70/70 [==============================] - 1s 9ms/step - loss: 1673736320.0000 - mean_squared_error: 1673736320.0000
Epoch 14/100
70/70 [==============================] - 1s 9ms/step - loss: 1650504448.0000 - mean_squared_error: 1650504448.0000
Epoch 15/100
70/70 [==============================] - 1s 9ms/step - loss: 1633853568.0000 - mean_squared_error: 1633853568.0000
Epoch 16/100
70/70 [==============================] - 1s 8ms/step - loss: 1620827264.0000 - mean_squared_error: 1620827264.0000
Epoch 17/100
70/70 [==============================] - 1s 8ms/step - loss: 1613732736.0000 - mean_squared_error: 1613732736.0000
Epoch 18/100
70/70 [==============================] - 1s 7ms/step - loss: 1613732480.0000 - mean_squared_error: 1613732480.0000
Epoch 19/100
70/70 [==============================] - 1s 9ms/step - loss: 1596426112.0000 - mean_squared_error: 1596426112.0000
Epoch 20/100
70/70 [==============================] - 1s 9ms/step - loss: 1578971648.0000 - mean_squared_error: 15789

70/70 [==============================] - 1s 11ms/step - loss: 6337568256.0000 - mean_squared_error: 6337568256.0000
Epoch 6/100
557/557 [==============================] - 2s 3ms/step - loss: 6975784960.0000 - mean_squared_error: 6975784960.0000
--- Starting trial: run-99
{'num_units': 500, 'dropout': 0.2, 'optimizer': 'adam', 'batch_size': 64}
Epoch 1/100
70/70 [==============================] - 1s 11ms/step - loss: 6323554816.0000 - mean_squared_error: 6323554816.0000
Epoch 2/100
70/70 [==============================] - 1s 11ms/step - loss: 6181968384.0000 - mean_squared_error: 6181968384.0000
Epoch 3/100
70/70 [==============================] - 1s 11ms/step - loss: 6092390912.0000 - mean_squared_error: 6092390912.0000
Epoch 4/100
70/70 [==============================] - 1s 12ms/step - loss: 5975247360.0000 - mean_squared_error: 5975247360.0000
Epoch 5/100
70/70 [==============================] - 1s 11ms/step - loss: 5772382720.0000 - mean_squared_error: 5772382720.0000
Epoch 6/100
70

In [11]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 11654), started 11:31:12 ago. (Use '!kill 11654' to kill it.)